## Cleaning epoched data

In [1]:
import pathlib
import matplotlib

import mne
import os
import mne_bids

matplotlib.use('Qt5Agg')
mne.set_log_level('warning')

In [4]:
epochs = mne.read_epochs(pathlib.Path('out_data') / 'epochs_epo.fif')
epochs

<EpochsFIF |  318 events (all good), -0.299693 - 0.499488 sec, baseline [-0.299693, 0] sec, ~442.1 MB, data loaded,
 'Auditory/Left': 72
 'Auditory/Right': 72
 'Button': 16
 'Smiley': 15
 'Visual/Left': 73
 'Visual/Right': 70>

In [3]:
epochs.apply_baseline((None, 0))

<EpochsFIF |  318 events (all good), -0.299693 - 0.499488 sec, baseline [-0.299693, 0] sec, ~442.1 MB, data loaded,
 'Auditory/Left': 72
 'Auditory/Right': 72
 'Button': 16
 'Smiley': 15
 'Visual/Left': 73
 'Visual/Right': 70>

In [ ]:
epochs.plot()

## Reject artifacts based on channel signal amplitude

In [ ]:
reject_criteria = dict(mag=3000e-15,     # 3000 fT
                       grad=3000e-13,    # 3000 fT/cm
                       eeg=150e-6,       # 150 µV
                       eog=200e-6)       # 200 µV

flat_criteria = dict(mag=1e-15,          # 1 fT
                     grad=1e-13,         # 1 fT/cm
                     eeg=1e-6)           # 1 µV

In [ ]:
epochs.drop_bad(reject=reject_criteria, flat=flat_criteria)

In [ ]:
epochs.plot_drop_log()

Quite a number of Epochs were dropped due to EOG artifacts.

In [ ]:
epochs['Visual'].plot_image()

In [ ]:
epochs.plot_sensors(ch_type='eeg')

In [ ]:
epochs['Visual'].plot_image(picks='EEG 060')

## SSP

### Let's see if we can retain most the epochs, but still get rid of the EOG artifact. And while we're at it, the ECG artifacts too 😉

In [ ]:
bids_root = pathlib.Path('out_data/sample_BIDS')

bids_path = mne_bids.BIDSPath(subject='01',
                              session='01',
                              task='audiovisual',
                              run='01',
                              datatype='meg',
                              root=bids_root)

raw = mne_bids.read_raw_bids(bids_path)
raw.load_data()
raw.filter(l_freq=0.1, h_freq=40)

ecg_projs, ecg_events = mne.preprocessing.compute_proj_ecg(raw, n_grad=1, n_mag=1, n_eeg=0,
                                                           average=True)

eog_projs, eog_events = mne.preprocessing.compute_proj_eog(raw, n_grad=1, n_mag=1,
                                                           n_eeg=1, average=True)

In [ ]:
eog_projs

In [ ]:
projs = eog_projs + ecg_projs
projs

In [ ]:
epochs.add_proj(projs)
epochs.plot()

In [ ]:
epochs_cleaned = epochs.copy().apply_proj()

epochs_cleaned['Visual'].plot_image(picks= 'eeg')
epochs_cleaned['Visual'].plot_image(picks='EEG 060')

## ICA

First, start with the **raw data** again and apply a 1.0 Hz high-pass filter, which is advantegeous for ICA performance.

In [5]:
bids_root = pathlib.Path('out_data/sample_BIDS')

bids_path = mne_bids.BIDSPath(subject='01',
                              session='01',
                              task='audiovisual',
                              run='01',
                              datatype='meg',
                              root=bids_root)

raw = mne_bids.read_raw_bids(bids_path)
raw.load_data()
raw.filter(l_freq=1, h_freq=40)  # High-pass with 1. Hz cut-off is recommended for ICA

<ipython-input-5-d7007c9a0285>:10: RuntimeWarning: The unit for channel(s) STI 001, STI 002, STI 003, STI 004, STI 005, STI 006, STI 014, STI 015, STI 016 has changed from V to NA.
  raw = mne_bids.read_raw_bids(bids_path)


<Raw | sub-01_ses-01_task-audiovisual_run-01_meg.fif, 376 x 166800 (277.7 s), ~481.8 MB, data loaded>

Read our epochs and extract which ones that were kept (**all** in our case, because we didn't apply any rejection procedure before saving the epochs in notebook 3; but this could be different in a real-world scenario, and you want to calculate ICA on the same set of epochs you're actually feeding into your analysis!)

In [6]:
epochs = mne.read_epochs(pathlib.Path('out_data') / 'epochs_epo.fif')
epochs_selection = epochs.selection
epochs_selection

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,
        80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,
        93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105,
       106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118,
       119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131,
       132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144,
       145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157,
       158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170,
       171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 18

Only keep the subset of events that corresponds to the retained epochs.

In [8]:
events, event_id = mne.events_from_annotations(raw)
events = events[epochs_selection]

Create epochs for ICA. All parameters should match **exactly** the ones of the epochs we intend to clean.

In [9]:
tmin = -0.3
tmax = 0.5
baseline = (None, 0)

epochs_ica = mne.Epochs(raw,
                        events=events,
                        event_id=event_id,
                        tmin=tmin,
                        tmax=tmax,
                        baseline=baseline,
                        preload=True)

### Finally, fit ICA!

In [10]:
epochs_ica.info

<Info | 23 non-empty values
 acq_pars: ACQch001 110113 ACQch002 110112 ACQch003 110111 ACQch004 110122 ...
 bads: 2 items (MEG 2443, EEG 053)
 ch_names: MEG 0113, MEG 0112, MEG 0111, MEG 0122, MEG 0123, MEG 0121, MEG ...
 chs: 204 GRAD, 102 MAG, 9 STIM, 60 EEG, 1 EOG
 custom_ref_applied: False
 description: acquisition (megacq) VectorView system at NMR-MGH
 dev_head_t: MEG device -> head transform
 dig: 146 items (3 Cardinal, 4 HPI, 61 EEG, 78 Extra)
 events: 1 item (list)
 experimenter: MEG
 file_id: 4 items (dict)
 highpass: 1.0 Hz
 hpi_meas: 1 item (list)
 hpi_results: 1 item (list)
 line_freq: 60
 lowpass: 40.0 Hz
 meas_date: 2002-12-03 19:01:10 UTC
 meas_id: 4 items (dict)
 nchan: 376
 proj_id: 1 item (ndarray)
 proj_name: test
 projs: PCA-v1: on, PCA-v2: on, PCA-v3: on
 sfreq: 600.6 Hz
 subject_info: 5 items (dict)
>

When running ICA the first argument of interest is the number of components. Normally, with EEG data the number of components we specify is the number of channels - 1 (that is, we subtract the reference), however, it is different with MEG data. We can also specify a number between 0 and 1, which can be though as a fraction. This way MNE will require as many components as needed to explain the variance/fraction we prespecified:

In [22]:
n_components = 0.8  # Should normally be higher, like 0.999!!
method = 'picard'
max_iter = 100  # Should normally be higher, like 500 or even 1000!!
fit_params = dict(fastica_it=5)
random_state = 42

# first create ICA object
ica = mne.preprocessing.ICA(n_components=n_components,
                            max_pca_components=300, # remove this param with mne 0.22 and above 
                            method=method,
                            max_iter=max_iter,
                            fit_params=fit_params,
                            random_state=random_state)
# run ICA 
ica.fit(epochs_ica)

<ipython-input-22-e926621bbb89>:13: DeprecationWarning: max_pca_components (300) is deprecated and will be removed in 0.23, use n_pca_components instead
  random_state=random_state)
/Users/christinadelta/environments/neuraldata/lib/python3.7/site-packages/picard/solver.py:216: UserWarning: Picard did not converge. Final gradient norm : 1.188e-05. Requested tolerance : 1e-07. Consider increasing the number of iterations or the tolerance.
  % (gradient_norm, tol))


<ICA | epochs decomposition, fit (picard): 152958 samples, 28 components, channels used: "mag"; "grad"; "eeg">

In [24]:
ica.plot_components(inst=epochs)

[<MNEFigure size 1950x1434 with 20 Axes>,
 <MNEFigure size 1950x992 with 8 Axes>]

### Detect ECG and EOG patterns

In [23]:
# create ecg epochs for ica componnets removal
ecg_epochs = mne.preprocessing.create_ecg_epochs(raw, reject=None,
                                                 baseline=(None, -0.2),
                                                 tmin=-0.5, tmax=0.5)
ecg_evoked = ecg_epochs.average()
ecg_inds, ecg_scores = ica.find_bads_ecg(
    ecg_epochs, method='ctps')

# create eog epochs for ica componnets removal
eog_epochs = mne.preprocessing.create_eog_epochs(raw, reject=None,
                                                 baseline=(None, -0.2),
                                                 tmin=-0.5, tmax=0.5)
eog_evoked = eog_epochs.average()
eog_inds, eog_scores = ica.find_bads_eog(
    eog_epochs)

# concantinate the ecg & eog components that are about to be removed 
components_to_exclude = ecg_inds + eog_inds
ica.exclude = components_to_exclude

In [25]:
ica.exclude

[0, 12, 1]

## Plot automated artifact detection scores

In [17]:
ica.plot_scores(ecg_scores)

<Figure size 1280x540 with 1 Axes>

In [16]:
ica.plot_scores(eog_scores)

<Figure size 1280x540 with 1 Axes>

## Plot ICA sources

In [18]:
ica.plot_sources(ecg_evoked)

<Figure size 1280x960 with 1 Axes>

In [21]:
ica.plot_sources(eog_evoked)

<Figure size 1280x960 with 1 Axes>

## Plot overlay of original and cleaned data

In [26]:
ica.plot_overlay(ecg_evoked)

<Figure size 1280x960 with 3 Axes>

In [27]:
ica.plot_overlay(eog_evoked)

<Figure size 1498x960 with 3 Axes>

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
         <li>Visualize artifact detection scores, ICA sources, and an overlay of original and cleaned data based on the EOG epochs.</li>
         <li>Visualize artifact detection scores, ICA sources, and an overlay of original and cleaned data based on the epochs we actually intend to analyze.</li>
    </ul>
</div>

### Apply ICA to the actual epochs 
We first applied ICA to epochs created specifically to look at which components are to be excluded. However, we are not going to use these ICA epochs that we. created earlier, we want to use the actual epochs created in notebook 2. So, the next step is to apply the ICA algorithm to the actual epochs:

In [30]:
epochs_cleaned = ica.apply(epochs.copy())

In [31]:
epochs_cleaned.plot()

<MNEBrowseFigure size 2560x1434 with 4 Axes>

### Plot the clean and unclean data side by side:

In [32]:
epochs_cleaned.plot(title='clean')
epochs.plot(title='not clean')

<MNEBrowseFigure size 2560x1434 with 4 Axes>

In [33]:
# save the clean data
epochs_cleaned.save(pathlib.Path('out_data') / 'epochs_cleaned_epo.fif', 
            overwrite=True)

<ipython-input-33-4240bb4d075e>:3: RuntimeWarning: This filename (out_data/epochs_epo_cleaned.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  overwrite=True)
